In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math
import re
import json

# 1. Append main category links to a file

In [3]:
main_url = "https://www2.hm.com/tr_tr/index.html"
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

urun_sonuclari = requests.get(main_url,headers=agent)
sonuc_soup = BeautifulSoup(urun_sonuclari.text, "html.parser")

In [4]:
link_list = sonuc_soup.find_all(class_="menu__sub-link") #ilgili class_'ta href'ler yer alıyor
cats = []
for i in link_list:
    cats.append(i.get('href').replace(".html",""))

Delete unnecessary category links afterwards

In [5]:
cats = [x for x in cats if("view-all" not in x and "hepsini-incele" not in x)]
len(cats)

350

## 2. Scraping

In [6]:
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
prod_list = []
'''
1.
ilgili kategorileri url'e ekleyerek json dict'lerden bilgileri çekiyoruz
2.
en fazla 10 sayfa (360 ürün) alıyor, değiştirebiliriz
total_urun bu sebeple kapatıldı
3.
dandik linkler ve indirimsiz ürünler için try/except kullanıyorum
'''
for cat in cats:
    url = f'https://www2.hm.com/{cat}/_jcr_content/main/productlisting.display.json?sort=stock&image-size=small&image=model&offset=36&page-size=450'
    try:
        r = requests.get(url,headers = agent)
        data = dict(r.json())
        #total_urun = int(data["total"])
        products =  data["products"]
        for prod in products:
            title = prod["title"]
            category = prod["category"]
            price = prod["price"]
            disc_price = prod["redPrice"]

            hm_dict = {
                'title':title,
                'category':category,
                'ust_category' : cat,
                'price':price,
                'disc_price':disc_price
            }
            prod_list.append(hm_dict)
    except:
        pass
hm_prods = pd.DataFrame(prod_list)

#  Format Change

In [7]:
hm_prods["disc_price"]= hm_prods.apply(lambda x: x['price'] if x["disc_price"] == "" else x['disc_price'], axis=1)
hm_prods["price"] = hm_prods["price"].apply(lambda x: x.replace(" TL",""))
hm_prods["disc_price"] = hm_prods["disc_price"].apply(lambda x: x.replace(" TL",""))
hm_prods["price"] = [x.replace("\xa0","") for x in hm_prods["price"]]
hm_prods["disc_price"] = [x.replace("\xa0","") for x in hm_prods["disc_price"]]
hm_prods["price"] = [x.replace(",",".") for x in hm_prods["price"]]
hm_prods["disc_price"] = [x.replace(",",".") for x in hm_prods["disc_price"]]
hm_prods["price"] = hm_prods.price.astype(float)
hm_prods["disc_price"] = hm_prods.disc_price.astype(float)
hm_prods

,title,category,ust_category,price,disc_price
0,Bukle Ceket,ladies_jacketscoats_jackets,/tr_tr/kadin/yeni-urunler/giysi,619.99,619.99
1,Üstte Fermuarlı Lastik Örgülü Kazak,ladies_cardigansjumpers_jumpers,/tr_tr/kadin/yeni-urunler/giysi,699.99,699.99
2,Yüksek Belli Şık Pantolon,ladies_trousers_highwaisted,/tr_tr/kadin/yeni-urunler/giysi,459.99,459.99
3,Kruvaze Oversize Ceket,ladies_blazers,/tr_tr/kadin/yeni-urunler/giysi,619.99,619.99
4,İspanyol Paçalı Fitilli Kadife Pantolon,ladies_trousers_flare,/tr_tr/kadin/yeni-urunler/giysi,619.99,619.99
...,...,...,...,...,...
29427,4’lü Keçe Bardak Altlığı,home_kitchen_servingware,/tr_tr/sustainability-at-hm/our-products/home,99.99,99.99
29428,Tüfte Kilim,home_rugs_largerugs,/tr_tr/sustainability-at-hm/our-products/home,3899.00,3899.00
29429,Keçe Kutu,home_decorations_decoraccessories,/tr_tr/sustainability-at-hm/our-products/home,619.99,619.99
29430,Tüfte Kilim,home_rugs_largerugs,/tr_tr/sustainability-at-hm/our-products/home,3099.00,3099.00


In [9]:
hm_prods.price.mean()         

422.0494122045937